In [1]:

#导入fastapi 包
from fastapi import FastAPI
#创建实例，名字为 app
app = FastAPI()

#创建路由
@app.get("/")
#路径操作函数
async def root():
    #返回内容
    return {"message": "Hello World"}

# 导入 FastAPI。
# 创建一个 app 实例。
# 编写一个路径操作装饰器（如 @app.get("/")）。
# 编写一个路径操作函数（如上面的 def root(): ...）。
# 运行开发服务器（如 uvicorn main:app --reload


fastapi常见的http方法：
    POST 请求 创建数据
    GET 请求 获取数据
    DELETE 请求 删除数据
    PUT 请求 更新数据
常见路由：
    @app.get('/')
    @app.get('/{id}')
    @app.post('/')
    @app.put('/{id}')
    @app.delete('/{id}')

In [ ]:
from fastapi import FastAPI

app = FastAPI()

#路径参数
@app.get("/items/{item_id}")
async def read_item(item_id):
    return {"item_id": item_id}

async def read_item(item_id: int):
    item_id 的类型声明为 int

In [ ]:
from enum import Enum

from fastapi import FastAPI


class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"


app = FastAPI()


@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}

    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the images"}

    return {"model_name": model_name, "message": "Have some residuals"}

预设值：
    预设值存入ModelName的类中，方便函数调用。

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

/files/{file_path:path}  ：参数名为 file_path，结尾部分的 :path 说明该参数应匹配路径。
本例中的 URL 是 /files//home/johndoe/myfile.txt。注意，files 和 home 之间要使用双斜杠（//），因为 path 参数会匹配到斜杠。

In [ ]:
from fastapi import FastAPI

app = FastAPI()

fake_items_db = [{"item_name": "Foo"}, {"item_name": "Bar"}, {"item_name": "Baz"}]


@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return fake_items_db[skip : skip + limit]

用 skip=0 和 limit=10 设定默认值。
上面代码中 访问 http://127.0.0.1:8000/items/  和 http://127.0.0.1:8000/items/?skip=0&limit=10 相同
如果访问 http://127.0.0.1:8000/items/?skip=20  即 skip=20，limit=10


In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_item(item_id: str, q: str | None = None):
    if q:
        return {"item_id": item_id, "q": q}
    return {"item_id": item_id}

# 查询参数 q 是可选的，默认值为 None

因为默认值为 = None，FastAPI 把 q 识别为可选参数。

FastAPI 不使用 Optional[str] 中的 Optional（只使用 str），但 Optional[str] 可以帮助编辑器发现代码中的错误。

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/users/{user_id}/items/{item_id}")
async def read_user_item(
    user_id: int, item_id: str, q: str | None = None, short: bool = False
):
    item = {"item_id": item_id, "owner_id": user_id}
    if q:
        item.update({"q": q})
    if not short:
        item.update(
            {"description": "This is an amazing item that has a long description"}
        )
    return item

为不是路径参数的参数声明默认值（至此，仅有查询参数），该参数就不是必选的了。

如果只想把参数设为可选，但又不想指定参数的值，则要把默认值设为 None。

如果要把查询参数设置为必选，就不要声明默认值：

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_user_item(item_id: str, needy: str):
    item = {"item_id": item_id, "needy": needy}
    return item

这里的查询参数 needy 是类型为 str 的必选查询参数。

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/{item_id}")
async def read_user_item(
    item_id: str, needy: str, skip: int = 0, limit: int | None = None
):
    item = {"item_id": item_id, "needy": needy, "skip": skip, "limit": limit}
    return item

needy，必选的 str 类型参数
skip，默认值为 0 的 int 类型参数
limit，可选的 int 类型参数

FastAPI 使用请求体从客户端（例如浏览器）向 API 发送数据。

请求体是客户端发送给 API 的数据。响应体是 API 发送给客户端的数据。

API 基本上肯定要发送响应体，但是客户端不一定发送请求体。

使用 Pydantic 模型声明请求体，能充分利用它的功能和优点。

说明

发送数据使用 POST（最常用）、PUT、DELETE、PATCH 等操作。

规范中没有定义使用 GET 发送请求体的操作，但不管怎样，FastAPI 也支持这种方式，只不过仅用于非常复杂或极端的用例。

我们不建议使用 GET，因此，在 Swagger UI 交互文档中不会显示有关 GET 的内容，而且代理协议也不一定支持 GET。

创建数据模型
把数据模型声明为继承 BaseModel 的类。

使用 Python 标准类型声明所有属性：

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    return item



与声明查询参数一样，包含默认值的模型属性是可选的，否则就是必选的。默认值为 None 的模型属性也是可选的。

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    return item

请求体参数的类型为 Item 模型。

仅使用 Python 类型声明，FastAPI 就可以：

以 JSON 形式读取请求体
（在必要时）把请求体转换为对应的类型
校验数据：
数据无效时返回错误信息，并指出错误数据的确切位置和内容
把接收的数据赋值给参数 item
把函数中请求体参数的类型声明为 Item，还能获得代码补全等编辑器支持
为模型生成 JSON Schema，在项目中所需的位置使用
这些概图是 OpenAPI 概图的部件，用于 API 文档 UI

使用模型
    在路径操作函数内部直接访问模型对象的属性：

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.post("/items/")
async def create_item(item: Item):
    item_dict = item.dict()
    if item.tax:
        price_with_tax = item.price + item.tax
        item_dict.update({"price_with_tax": price_with_tax})
    return item_dict

请求体 + 路径参数
FastAPI 支持同时声明路径参数和请求体。

FastAPI 能识别与路径参数匹配的函数参数，还能识别从请求体中获取的类型为 Pydantic 模型的函数参数。

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item):
    return {"item_id": item_id, **item.dict()}

请求体 + 路径参数 + 查询参数
FastAPI 支持同时声明请求体、路径参数和查询参数。

FastAPI 能够正确识别这三种参数，并从正确的位置获取数据。

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel


class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


app = FastAPI()


@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item, q: str | None = None):
    result = {"item_id": item_id, **item.dict()}
    if q:
        result.update({"q": q})
    return result

路径中声明了相同参数的参数，是路径参数
类型是（int、float、str、bool 等）单类型的参数，是查询参数
类型是 Pydantic 模型的参数，是请求体

查询参数和字符串校验

In [ ]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/items/")
async def read_items(q: str | None = None):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

查询参数 q 的类型为 str，默认值为 None，因此它是可选的。

In [ ]:
额外的校验
我们打算添加约束条件：即使 q 是可选的，但只要提供了该参数，则该参数值不能超过50个字符的长度
为此，首先从 fastapi 导入 Query

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Union[str, None] = Query(default=None, max_length=50)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

使用 Query 作为默认值¶
现在，将 Query 用作查询参数的默认值，并将它的 max_length 参数设置为 50：

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Union[str, None] = Query(default=None, max_length=50)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

由于我们必须用 Query(default=None) 替换默认值 None，Query 的第一个参数同样也是用于定义默认值。

所以：q: Union[str, None] = Query(default=None) 使得参数可选，等同于 q: str = None

In [ ]:
添加更多校验
还可以添加 min_length 参数：

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(default=None, min_length=3, max_length=50),
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

添加最大，最小的校验

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(
        default=None, min_length=3, max_length=50, pattern="^fixedquery$"
    ),
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

这个指定的正则表达式通过以下规则检查接收到的参数值：

^：以该符号之后的字符开头，符号之前没有字符。
fixedquery: 值精确地等于 fixedquery。
$: 到此结束，在 fixedquery 之后没有更多字符。


但是，一旦你需要用到并去学习它们时，请了解你已经可以在 FastAPI 中直接使用它们。

默认值
你可以向 Query 的第一个参数传入 None 用作查询参数的默认值，以同样的方式你也可以传递其他默认值。

假设你想要声明查询参数 q，使其 min_length 为 3，并且默认值为 fixedquery：



声明为必需参数
当我们不需要声明额外的校验或元数据时，只需不声明默认值就可以使 q 参数成为必需参数，例如：q: str  代替  q: Union[str, None] = None  但是现在我们正在用 Query 声明它，例如：q: Union[str, None] = Query(default=None, min_length=3)  例如：

In [ ]:
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: str = Query(default="fixedquery", min_length=3)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

在使用 Query 且需要声明一个值是必需的时，只需不声明默认参数：

In [ ]:
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: str = Query(min_length=3)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

使用省略号(...)声明必需参数¶
有另一种方法可以显式的声明一个值是必需的，即将默认参数的默认值设为 ... ：

In [ ]:
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: str = Query(default=..., min_length=3)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

这将使 FastAPI 知道此查询参数是必需的。

使用None声明必需参数¶
你可以声明一个参数可以接收None值，但它仍然是必需的。这将强制客户端发送一个值，即使该值是None。

为此，你可以声明None是一个有效的类型，并仍然使用default=...：

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Union[str, None] = Query(default=..., min_length=3)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

使用Pydantic中的Required代替省略号(...)¶
如果你觉得使用 ... 不舒服，你也可以从 Pydantic 导入并使用 Required：

In [ ]:
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: str = Query(default=..., min_length=3)):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

查询参数列表 / 多个值¶
当你使用 Query 显式地定义查询参数时，你还可以声明它去接收一组值，或换句话来说，接收多个值。

例如，要声明一个可在 URL 中出现多次的查询参数 q，你可以这样写：

In [ ]:
from typing import List, Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Union[List[str], None] = Query(default=None)):
    query_items = {"q": q}
    return query_items




# http://localhost:8000/items/?q=foo&q=bar

In [ ]:
from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: list = Query(default=[])):
    query_items = {"q": q}
    return query_items

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(default=None, title="Query string", min_length=3),
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(
        default=None,
        title="Query string",
        description="Query string for the items to search in the database that have a good match",
        min_length=3,
    ),
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

别名参数
假设你想要查询参数为 item-query。

像下面这样：


http://127.0.0.1:8000/items/?item-query=foobaritems
但是 item-query 不是一个有效的 Python 变量名称。

最接近的有效名称是 item_query。

但是你仍然要求它在 URL 中必须是 item-query...

这时你可以用 alias 参数声明一个别名，该别名将用于在 URL 中查找查询参数值：

In [ ]:
from typing import Union

from fastapi import FastAPI, Query

app = FastAPI()


@app.get("/items/")
async def read_items(q: Union[str, None] = Query(default=None, alias="item-query")):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

弃用参数¶
现在假设你不再喜欢此参数。

你不得不将其保留一段时间，因为有些客户端正在使用它，但你希望文档清楚地将其展示为已弃用。

那么将参数 deprecated=True 传入 Query：

In [ ]:
from typing import Union

from fastapi import FastAPI, Query
import uvicorn

app = FastAPI()


@app.get("/items/")
async def read_items(
    q: Union[str, None] = Query(
        default=None,
        alias="item-query",
        title="Query string",
        description="Query string for the items to search in the database that have a good match",
        min_length=3,
        max_length=50,
        pattern="^fixedquery$",
        deprecated=True,
    ),
):
    results = {"items": [{"item_id": "Foo"}, {"item_id": "Bar"}]}
    if q:
        results.update({"q": q})
    return results

uvicorn.run(app, host="127.0.0.1", port=8000)

总结¶
你可以为查询参数声明额外的校验和元数据。

通用的校验和元数据：

alias
title
description
deprecated
特定于字符串的校验：

min_length
max_length
regex
在这些示例中，你了解了如何声明对 str 值的校验。